In [24]:
import pandas as pd
import numpy as np
import os
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string

In [15]:
path = os.path.join(os.path.dirname(os.getcwd()),'raw_data')
df = pd.read_csv(os.path.join(path,'bbc_final_df.csv'))

In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
df

,recipe_title,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
0,Easy creamed spinach,vegetarian,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
1,Easy creamed spinach,gluten-free,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
2,Simple coconut & bean soup,vegetarian,4.8,10,20,22.0,Easy,Serves 4,This vegetarian meal in a bowl shows how somet...,"1 tbsp ,sunflower oil, ½ bunch spring onions,,...","sunflower oil, ½ bunch spring onions, red pepp...",caribbean,"['sunflower', 'oil', 'spring', 'onions', 'pepp..."
3,Free-form Florentine pie,vegetarian,3.8,10,35,14.0,Easy,Serves 4,You're just six ingredients away from an impre...,"2 tsp ,olive oil, 400g ,bag fresh spinach, 250...","olive oil, bag fresh spinach, tub ricotta, gra...",italian,"['olive', 'oil', 'spinach', 'ricotta', 'nutmeg..."
4,Fresh herb sauce,vegetarian,4.0,5,5,1.0,Easy,Serves 8,This fresh green purée is a light alternative ...,"85g ,watercress, 50g ,parsley, 50g ,walnut, 15...","watercress, parsley, walnut, olive oil, sugar,",italian,"['watercress', 'parsley', 'walnut', 'olive', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,Easy noodles,n,3.8,20,n,23.0,Easy,Serves 4,This simple version of a Malaysian classic is ...,"500ml ,hot chicken stock, or vegetable stock...","coconut milk, green or red Thai curry paste, ...",vietnamese,"['coconut', 'milk', 'green', 'thai', 'curry', ..."
10879,Rice noodle & turkey salad with lime-chilli dr...,n,4.3,20,10,10.0,Easy,Serves 4,Lean turkey mince is low in fat - serve with n...,"100g ,dried thin rice noodle, 1 ,red chilli,, ...","red chilli, carrots, small red onion, handful...",vietnamese,"['chilli', 'carrot', 'onion', 'mint', 'fish', ..."
10880,Cevapcici with red pepper rice,n,n,25,30,n,Easy,Serves 4,These flavourful spiced beef meatballs with to...,"500g ,beef mince, 2 ,onions,, finely chopped, ...","beef mince, onions, garlic cloves, sweet papri...",balkan,"['beef', 'mince', 'onion', 'garlic', 'cloves',..."
10881,Pork & cabbage with bread dumplings,n,n,40,3 hrs and 30,n,Easy,Serves 4 - 6,Try this simple dish of succulent slow-cooked ...,"1¼kg ,pork collar, 1 tbsp ,cumin seeds,, fresh...","pork collar, cumin seeds, garlic cloves, onion...",czech,"['pork', 'collar', 'cumin', 'garlic', 'cloves'..."


In [20]:
def model_specific_preprocessing(tokenized_sentence):
    stop_words = set(stopwords.words('english'))
    no_stopwords = [w for w in tokenized_sentence if not w in stop_words]
    verb_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "v") for tok in no_stopwords]
    noun_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "n")for tok in verb_lemmatized]
    ingredient_string = ' '.join(noun_lemmatized)
    return ingredient_string

def ingredients_list(sample_txt):
    elements = []
    for punc in string.punctuation:
        sample_txt = sample_txt.replace(punc,'')
    for word in sample_txt.split():
        elements.append(str(word))
    return elements

In [18]:
def k_means(bbc_final_df, clusters=10, min_df=0.005, max_df=0.99):
    """Returns the df inputted with an additional column with their respective cluster.
    Ensure that the ingredients columns is a string of a list called 'final_ingredients'.
    I.E row 1, col ingredient is '[bread,cheese,pasta,onions]'"""

    # Step 1: Filter out recipes with no names & remove duplicates, keeping dietary info
    filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
    grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
    grouped_df.rename(columns = {'dietary':'combined'}, inplace=True)
    merged_df = grouped_df.merge(filtered_df,how='left',on='recipe_title')
    dropped_df = merged_df.drop(columns = 'dietary')
    df = dropped_df.drop_duplicates('recipe_title')

    # Step 2: Convert the final ingredients into a "sentence" for vectorising
    df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
    df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))

    # Step 3: Convert ingredients sentence into a bag-of-words representation
    vectorizer = CountVectorizer(min_df=min_df, max_df = max_df)
    counted_words = pd.DataFrame(vectorizer.fit_transform(df['clean_text']).toarray(), columns = vectorizer.get_feature_names_out())

    # Step 4: Apply KMeans clustering
    kmeans = KMeans(n_clusters=clusters, random_state=0)
    df['cluster'] = kmeans.fit_predict(counted_words)

    return df

In [25]:
df1 = k_means(df)

/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_77267/3738140175.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_77267/3738140175.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_77267/3738140175.py:24: SettingWith

In [29]:
df1.tail()

,recipe_title,combined,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients,ingredients_list,clean_text,cluster
10580,Wild rice & feta salad,n,4.8,5,25,12.0,Easy,Serves 4,Perfect for a casual get together and great fo...,"250g ,basmati and wild rice, 400g ,can chickpe...","basmati and wild rice, pack dried cranberry, r...",indian,"['basmati', 'rice', 'cranberry', 'onion', 'gar...","[basmati, rice, cranberry, onion, garlic, clov...",basmati rice cranberry onion garlic cleave oli...,4
10631,Yellow lentil & coconut curry with cauliflower,vegetarian,3.7,15,50,36.0,Easy,Serves 4,This healthy veggie curry is diet-friendly and...,"1 tbsp ,vegetable oil, 1 ,onion,, thinly slice...","onion, garlic cloves, ginger, unsweetened desi...",indian,"['onion', 'garlic', 'cloves', 'ginger', 'desic...","[onion, garlic, cloves, ginger, desiccated, co...",onion garlic clove ginger desiccate coconut ca...,6
10632,Yellow pepper rice,vegetariandairy-free,4.6,5,20,18.0,Easy,Serves 4,"Perfect with salmon or white fish, this makes ...","1 tbsp ,olive oil, 1 ,onion,, finely chopped, ...","olive oil, onion, red pepper, basmati rice, tu...",indian,"['olive', 'oil', 'onion', 'pepper', 'basmati',...","[olive, oil, onion, pepper, basmati, rice, tur...",olive oil onion pepper basmati rice turmeric a...,5
10634,Yogurt & almond chicken curry,n,2.0,15,40,15.0,Easy,Serves 6,Our ultimate Friday night food - just add naan...,"2 ,green chillies,, roughly chopped, 100g ,gro...","green chillies, ground almonds, thumb-size pie...",indian,"['green', 'chillies', 'almonds', 'ginger', 'ga...","[green, chillies, almonds, ginger, garlic, clo...",green chilli almond ginger garlic clove cream ...,0
10636,Yogurt & spice crusted roast potatoes,vegetarian,4.2,10,30,6.0,Easy,Serves 2,A superhealthy side dish which goes especially...,"500g ,baking potatoes,, quartered, 250g ,natur...","baking potatoes, natural set yogurt, 1 large g...",indian,"['potatoes', 'natural', 'set', 'yogurt', 'gree...","[potatoes, natural, set, yogurt, green, chilli...",potato natural set yogurt green chilli ginger ...,4


In [27]:
df1 = df1[df1['preference']=='indian']

In [28]:
df1

,recipe_title,combined,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients,ingredients_list,clean_text,cluster
42,Ajwain puri,vegetarianvegandairy-free,n,20,20,n,n,Serves 4,Enjoy these flatbread as part on an Indian fea...,"300g ,chapatti flour, ½ tsp ,ajwain,, or cumin...","sunflower oil,",indian,"['sunflower', 'oil']","[sunflower, oil]",sunflower oil,2
45,Akhni fulab (lamb pilau),gluten-free,n,1 hr and 10,1 hr and 30,2.0,Easy,Serves 8-10,Try a Bangladeshi version of what’s similarly ...,"1kg diced ,lamb, (shoulder or leg) , 2 large ...","lamb, onions, garlic cloves, ginger, cinnamon ...",indian,"['lamb', 'onion', 'garlic', 'cloves', 'ginger'...","[lamb, onion, garlic, cloves, ginger, cinnamon...",lamb onion garlic clove ginger cinnamon turmer...,0
46,Akoori (Indian scrambled eggs),vegetariangluten-free,4.0,10,8,31.0,Easy,Serves 4,"Reader Jango Gazdar shares a spicy, flavour-pa...","1 tbsp ,butter, 1 ,small red onion,, finely ch...","butter, small red onion, green or red chilli, ...",indian,"['butter', 'onion', 'green', 'chilli', 'garlic...","[butter, onion, green, chilli, garlic, cloves,...",butter onion green chilli garlic clove cumin g...,0
63,All-in-one posh lamb balti,n,4.8,35,4 hrs and 20,20.0,Easy,Serves 5 - 6,Feed a crowd with this gorgeous slow-cooked cu...,"5-6 lamb shanks, 3-4 onions,, halved and sli...","5-6 lamb shanks, 3-4 onions, fresh ginger, ...",indian,"['lamb', 'onions', 'ginger', 'garlic', 'cloves...","[lamb, onions, ginger, garlic, cloves, cans, t...",lamb onion ginger garlic clove can tomato gara...,0
82,Aloo chaat,vegangluten-freeegg-free,n,15,30,n,Easy,Serves 4-6,Enjoy a popular Indian festival snack. These l...,"750g ,potatoes, (Maris Piper or King Edward),...","potatoes, red onion, green chillies, ground cu...",indian,"['potato', 'onion', 'green', 'chillies', 'cumi...","[potato, onion, green, chillies, cumin, kashmi...",potato onion green chilli cumin kashmiri chill...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10580,Wild rice & feta salad,n,4.8,5,25,12.0,Easy,Serves 4,Perfect for a casual get together and great fo...,"250g ,basmati and wild rice, 400g ,can chickpe...","basmati and wild rice, pack dried cranberry, r...",indian,"['basmati', 'rice', 'cranberry', 'onion', 'gar...","[basmati, rice, cranberry, onion, garlic, clov...",basmati rice cranberry onion garlic cleave oli...,4
10631,Yellow lentil & coconut curry with cauliflower,vegetarian,3.7,15,50,36.0,Easy,Serves 4,This healthy veggie curry is diet-friendly and...,"1 tbsp ,vegetable oil, 1 ,onion,, thinly slice...","onion, garlic cloves, ginger, unsweetened desi...",indian,"['onion', 'garlic', 'cloves', 'ginger', 'desic...","[onion, garlic, cloves, ginger, desiccated, co...",onion garlic clove ginger desiccate coconut ca...,6
10632,Yellow pepper rice,vegetariandairy-free,4.6,5,20,18.0,Easy,Serves 4,"Perfect with salmon or white fish, this makes ...","1 tbsp ,olive oil, 1 ,onion,, finely chopped, ...","olive oil, onion, red pepper, basmati rice, tu...",indian,"['olive', 'oil', 'onion', 'pepper', 'basmati',...","[olive, oil, onion, pepper, basmati, rice, tur...",olive oil onion pepper basmati rice turmeric a...,5
10634,Yogurt & almond chicken curry,n,2.0,15,40,15.0,Easy,Serves 6,Our ultimate Friday night food - just add naan...,"2 ,green chillies,, roughly chopped, 100g ,gro...","green chillies, ground almonds, thumb-size pie...",indian,"['green', 'chillies', 'almonds', 'ginger', 'ga...","[green, chillies, almonds, ginger, garlic, clo...",green chilli almond ginger garlic clove cream ...,0


# dsfjadsfasd

In [39]:
ls

all_cuisine_df.csv  data.py             train.json
bbc_final_df.csv    non_brit.csv


In [42]:
dff = pd.read_csv('bbc_final_df.csv', index_col=0)

In [43]:
bbc_final_df = dff.copy()
bbc_final_df

,recipe_title,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
0,Easy creamed spinach,vegetarian,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
1,Easy creamed spinach,gluten-free,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
2,Simple coconut & bean soup,vegetarian,4.8,10,20,22.0,Easy,Serves 4,This vegetarian meal in a bowl shows how somet...,"1 tbsp ,sunflower oil, ½ bunch spring onions,,...","sunflower oil, ½ bunch spring onions, red pepp...",caribbean,"['sunflower', 'oil', 'spring', 'onions', 'pepp..."
9,Free-form Florentine pie,vegetarian,3.8,10,35,14.0,Easy,Serves 4,You're just six ingredients away from an impre...,"2 tsp ,olive oil, 400g ,bag fresh spinach, 250...","olive oil, bag fresh spinach, tub ricotta, gra...",italian,"['olive', 'oil', 'spinach', 'ricotta', 'nutmeg..."
28,Fresh herb sauce,vegetarian,4.0,5,5,1.0,Easy,Serves 8,This fresh green purée is a light alternative ...,"85g ,watercress, 50g ,parsley, 50g ,walnut, 15...","watercress, parsley, walnut, olive oil, sugar,",italian,"['watercress', 'parsley', 'walnut', 'olive', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18662,Easy noodles,n,3.8,20,n,23.0,Easy,Serves 4,This simple version of a Malaysian classic is ...,"500ml ,hot chicken stock, or vegetable stock...","coconut milk, green or red Thai curry paste, ...",vietnamese,"['coconut', 'milk', 'green', 'thai', 'curry', ..."
18663,Rice noodle & turkey salad with lime-chilli dr...,n,4.3,20,10,10.0,Easy,Serves 4,Lean turkey mince is low in fat - serve with n...,"100g ,dried thin rice noodle, 1 ,red chilli,, ...","red chilli, carrots, small red onion, handful...",vietnamese,"['chilli', 'carrot', 'onion', 'mint', 'fish', ..."
18664,Cevapcici with red pepper rice,n,n,25,30,n,Easy,Serves 4,These flavourful spiced beef meatballs with to...,"500g ,beef mince, 2 ,onions,, finely chopped, ...","beef mince, onions, garlic cloves, sweet papri...",balkan,"['beef', 'mince', 'onion', 'garlic', 'cloves',..."
18665,Pork & cabbage with bread dumplings,n,n,40,3 hrs and 30,n,Easy,Serves 4 - 6,Try this simple dish of succulent slow-cooked ...,"1¼kg ,pork collar, 1 tbsp ,cumin seeds,, fresh...","pork collar, cumin seeds, garlic cloves, onion...",czech,"['pork', 'collar', 'cumin', 'garlic', 'cloves'..."


In [44]:
filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
filtered_df

,recipe_title,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
0,Easy creamed spinach,vegetarian,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
1,Easy creamed spinach,gluten-free,4.6,2,6,5.0,Easy,Serves 2,"Double cream, Parmesan cheese and a pinch of n...","1 tbsp ,butter, 200g ,spinach, 75ml ,double cr...","butter, spinach, double cream, grated parmesan...",british,"['butter', 'spinach', 'cream', 'parmesan', 'nu..."
2,Simple coconut & bean soup,vegetarian,4.8,10,20,22.0,Easy,Serves 4,This vegetarian meal in a bowl shows how somet...,"1 tbsp ,sunflower oil, ½ bunch spring onions,,...","sunflower oil, ½ bunch spring onions, red pepp...",caribbean,"['sunflower', 'oil', 'spring', 'onions', 'pepp..."
9,Free-form Florentine pie,vegetarian,3.8,10,35,14.0,Easy,Serves 4,You're just six ingredients away from an impre...,"2 tsp ,olive oil, 400g ,bag fresh spinach, 250...","olive oil, bag fresh spinach, tub ricotta, gra...",italian,"['olive', 'oil', 'spinach', 'ricotta', 'nutmeg..."
28,Fresh herb sauce,vegetarian,4.0,5,5,1.0,Easy,Serves 8,This fresh green purée is a light alternative ...,"85g ,watercress, 50g ,parsley, 50g ,walnut, 15...","watercress, parsley, walnut, olive oil, sugar,",italian,"['watercress', 'parsley', 'walnut', 'olive', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18662,Easy noodles,n,3.8,20,n,23.0,Easy,Serves 4,This simple version of a Malaysian classic is ...,"500ml ,hot chicken stock, or vegetable stock...","coconut milk, green or red Thai curry paste, ...",vietnamese,"['coconut', 'milk', 'green', 'thai', 'curry', ..."
18663,Rice noodle & turkey salad with lime-chilli dr...,n,4.3,20,10,10.0,Easy,Serves 4,Lean turkey mince is low in fat - serve with n...,"100g ,dried thin rice noodle, 1 ,red chilli,, ...","red chilli, carrots, small red onion, handful...",vietnamese,"['chilli', 'carrot', 'onion', 'mint', 'fish', ..."
18664,Cevapcici with red pepper rice,n,n,25,30,n,Easy,Serves 4,These flavourful spiced beef meatballs with to...,"500g ,beef mince, 2 ,onions,, finely chopped, ...","beef mince, onions, garlic cloves, sweet papri...",balkan,"['beef', 'mince', 'onion', 'garlic', 'cloves',..."
18665,Pork & cabbage with bread dumplings,n,n,40,3 hrs and 30,n,Easy,Serves 4 - 6,Try this simple dish of succulent slow-cooked ...,"1¼kg ,pork collar, 1 tbsp ,cumin seeds,, fresh...","pork collar, cumin seeds, garlic cloves, onion...",czech,"['pork', 'collar', 'cumin', 'garlic', 'cloves'..."


In [67]:
filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
grouped_df = filtered_df.groupby('recipe_title')['dietary'].apply(lambda x: ', '.join(x)).reset_index().rename(columns = {'dietary':'combined'})
#grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
merged_df = grouped_df.merge(filtered_df, how='left',on='recipe_title')
dropped_df = merged_df.drop(columns = 'dietary')
df = dropped_df.drop_duplicates('recipe_title')

In [68]:
df

,recipe_title,combined,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
0,'Panforte' pies,n,4.7,15,30,3.0,Easy,Makes 12,The traditional Italian dessert is turned into...,"140g ,mixed shelled and blanched nut, such as ...","plain flour, butter, dark brown muscovado suga...",italian,"['flour', 'butter', 'muscovado', 'sugar', 'map..."
1,10-minute couscous salad,vegetarian,4.7,10,n,208.0,Easy,Serves 2,This makes a great lunchbox filler for a day o...,"100g ,couscous, 200ml ,hot low salt vegetable ...","couscous, spring onions, red pepper, feta chee...",mediterranean,"['couscous', 'spring', 'onions', 'pepper', 'fe..."
2,10-minute pad Thai,n,3.9,5,5,31.0,Easy,Serves 2,A delicious one-pan dinner - who needs takeaways?,"200g ,raw prawns, small pack coriander,, stalk...","raw prawns, beansprouts, egg, juice 1 lime, fi...",thai,"['prawns', 'beansprouts', 'egg', 'lime', 'fish..."
3,10-minute steak & blue cheese wrap,n,3.0,10,n,1.0,Easy,Serves 1,"A satisfying, meaty snack for a quick bite","1 tbsp ,olive oil, 1 x 140g/5oz ,sirloin steak...","olive oil, sirloin steak, small red onion, re...",american,"['olive', 'oil', 'steak', 'onion', 'pepper', '..."
4,10-minute steak & noodle supper,n,3.0,10,n,3.0,Easy,Serves 2,A super-speedy beef and egg noodle dish – and ...,"200g ,rump steak, 200g ,pak choi, 1 ,red peppe...","pak choi, red pepper, sunflower oil,",chinese,"['pak', 'choi', 'pepper', 'sunflower', 'oil']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,Zingy salmon & brown rice salad,healthy,4.0,15,25,43.0,Easy,Serves 3 - 4,This summery recipe has the ideal combination ...,"200g ,brown basmati rice, 200g ,frozen soya be...","brown basmati rice, frozen soya beans, salmon ...",british,"['basmati', 'rice', 'soya', 'beans', 'salmon',..."
10669,Zombie fingers,n,4.0,30,n,4.0,Easy,Serves about 15,These freaky edible fingers are naturally swee...,"250g ,stoned dates, 100g ,dark chocolate,, cho...","stoned dates, dark chocolate, smooth peanut bu...",british,"['chocolate', 'peanut', 'butter', 'almonds']"
10670,Zoolbia (lacy saffron fritters),vegetarian,n,5,25,n,Easy,Makes 14-16,Make these moreish Persian fritters for a cele...,"175g ,plain flour, 1 x 7g sachet ,fast-acting...","plain flour, icing sugar,",persian,"['flour', 'sugar']"
10671,‘Butter pie’ with apples & cheese,vegetarian,4.0,20,1 hr and 30,4.0,More effort,Cuts into 10,This hearty potato pie hails from Lancashire a...,"800g ,potato, (we used Desiree), sliced into 5...","unsalted butter, large (not mild) onions, smal...",english,"['butter', 'not', 'onions', 'thyme', 'coxs', '..."


In [73]:
set(df.combined.tolist())

{'dairy-free',
 'dairy-free, low-calorie, low-sugar, high-protein',
 'dairy-free, low-sugar',
 'egg-free',
 'egg-free, low-calorie, low-sugar, low-fat',
 'gluten-free',
 'gluten-free, dairy-free',
 'gluten-free, dairy-free, egg-free',
 'gluten-free, egg-free',
 'gluten-free, healthy',
 'gluten-free, healthy, dairy-free',
 'gluten-free, healthy, dairy-free, egg-free',
 'gluten-free, healthy, dairy-free, low-calorie, low-fat',
 'gluten-free, healthy, egg-free',
 'gluten-free, healthy, high-protein',
 'gluten-free, healthy, keto',
 'gluten-free, healthy, low-calorie, high-protein',
 'gluten-free, high-fibre',
 'gluten-free, high-protein',
 'gluten-free, keto',
 'gluten-free, low-sugar, keto, low-carb',
 'gluten-free, nut-free',
 'gluten-free, nut-free, dairy-free, egg-free',
 'gluten-free, nut-free, egg-free',
 'gluten-free, nut-free, egg-free, high-protein',
 'gluten-free, nut-free, egg-free, low-sugar, high-protein, low-carb',
 'gluten-free, nut-free, healthy',
 'gluten-free, nut-free, 

In [71]:
df.tail(20)

,recipe_title,combined,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
10653,Zesty lamb chops with crushed kidney beans,n,n,15,20,6.0,Easy,Serves 4,"Perk up lamb chops with lemon zest, then serve...","8 ,lamb chops, 2 tbsp ,olive oil, juice and ze...","lamb chops, olive oil, lemon, red chillies, ga...",mexican,"['lamb', 'olive', 'oil', 'lemon', 'chillies', ..."
10654,Zesty lentil & haddock pilaf,dairy-free,2.6,4,16,3.0,Easy,Serves 4,"This delicious, healthy dish is easily doubled...","250g ,easy-cook basmati rice, 3 ,red onions,, ...","easy-cook basmati rice, red onions, olive oil,...",mediterranean,"['easycook', 'basmati', 'rice', 'onions', 'oli..."
10655,Zesty mincemeat,n,n,15,n,6.0,Easy,Makes 1.3kg,This coarse traditional mincemeat doesn't requ...,"500g ,bag standard mixed dried fruit, 100g ,ch...","light muscovado sugar, zest and juice 1 orange...",english,"['muscovado', 'sugar', 'orange', 'lemon', 'coa..."
10656,Zesty orange marzipan,n,4.6,45,n,15.0,Easy,n,Homemade marzipan out does shop-bought every t...,"175g ,caster sugar, 175g ,icing sugar,, plus e...","caster sugar, icing sugar, ground almond, oran...",british,"['sugar', 'sugar', 'almond', 'orange', 'egg', ..."
10657,Zesty pomegranate rice,vegetarian,4.0,10,15,1.0,Easy,Serves 4,This fragrant dish adds an exotic flavour to p...,"500ml ,vegetable stock, 250g ,basmati rice, 10...","basmati rice, 100g or 120g tub pomegranate, co...",middle-eastern,"['basmati', 'rice', 'pomegranate', 'coriander'..."
10658,Zesty rice noodle salad,healthy,3.9,25,5,7.0,Easy,Serves 4,Wake up your taste buds with this superhealthy...,"250g ,vermicelli, rice noodles, 4 ,carrots,, c...","carrots, large red onion, juice 6 limes, caste...",asian,"['carrot', 'onion', 'limes', 'sugar', 'fish', ..."
10659,Zesty roast salmon & cod,healthy,3.9,45,n,12.0,Easy,Serves 8,Roasting is a foolproof way of cooking fish. T...,"800g ,thick skinless boneless salmon fillet,,...","thick skinless boneless salmon fillet, thick ...",british,"['salmon', 'fillet', 'cod', 'olive', 'oil', 'o..."
10660,Zesty salmon with roasted beets & spinach,gluten-free,4.1,10,1 hr,9.0,Easy,Serves 2,This fresh and filling salmon supper is packed...,"4 ,small fresh beetroots,, about 200g, 1 ½ tbs...","small fresh beetroots, rapeseed oil, coriander...",british,"['beetroots', 'rapeseed', 'oil', 'coriander', ..."
10661,Zesty spring fish,n,4.1,10,15,11.0,Easy,Serves 2,Perk up white fish with soy sauce and ginger -...,"2 ,large fillets white fish,, about 200g/8oz e...","2 tbsp soy sauce, garlic cloves, small knob ...",british,"['soy', 'sauce', 'garlic', 'cloves', 'ginger']"
10662,Zesty strawberries with Cointreau,n,4.0,5,n,9.0,Easy,n,Make the most of the fresh taste of British st...,"500g ,strawberry,, hulled halved or quartered,...","strawberry, orange, icing sugar, mint,",british,"['strawberry', 'orange', 'sugar', 'mint']"


In [83]:
df['combined'].replace('n','None')

0               None
1         vegetarian
2               None
3               None
4               None
            ...     
10668        healthy
10669           None
10670     vegetarian
10671     vegetarian
10672    gluten-free
Name: combined, Length: 9045, dtype: object

In [74]:
cuisine_pref = 'american'
diet_pref = 'vegetarian'

In [79]:
if cuisine_pref and diet_pref == 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    # st.write(star_sorted_df)
    print(star_sorted_df)

if cuisine_pref == 'Select' and diet_pref != 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df[star_sorted_df['combined'].contains(diet_pref)]
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    print(star_sorted_df)

    # st.write(star_sorted_df)
if diet_pref == 'Select'and cuisine_pref != 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df[star_sorted_df['preference']==cuisine_pref]
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    print(star_sorted_df)


In [78]:
star_sorted_df = processed_df[processed_df['stars']!='n']
star_sorted_df[star_sorted_df['preference']==cuisine_pref]
star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
star_sorted_df.reset_index(drop=True,inplace=True)
star_sorted_df.sort_values(by='stars', ascending=False)

NameError: name 'star_sorted_df' is not defined

In [86]:
df22 = pd.read_csv('bbc_final_df.csv')

In [93]:
star_sorted_df = df22.copy()
star_sorted_df=star_sorted_df[star_sorted_df['stars']!='n']

In [116]:
filtered_df = df22[df22['recipe_title'] != 'n']

grouped_df = filtered_df.groupby('recipe_title')['dietary'].apply(lambda x: ', '.join(x)).reset_index().rename(columns = {'dietary':'combined'})
# grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
# grouped_df.rename(columns = {'dietary':'combined'}, inplace=True)
merged_df = grouped_df.merge(filtered_df, how='left',on='recipe_title')
# dropped_df = merged_df.drop(columns = 'dietary')
df = merged_df.drop_duplicates('recipe_title')



In [117]:
df

,recipe_title,combined,Unnamed: 0,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
0,'Panforte' pies,n,17320,n,4.7,15,30,3.0,Easy,Makes 12,The traditional Italian dessert is turned into...,"140g ,mixed shelled and blanched nut, such as ...","plain flour, butter, dark brown muscovado suga...",italian,"['flour', 'butter', 'muscovado', 'sugar', 'map..."
1,10-minute couscous salad,vegetarian,2522,vegetarian,4.7,10,n,208.0,Easy,Serves 2,This makes a great lunchbox filler for a day o...,"100g ,couscous, 200ml ,hot low salt vegetable ...","couscous, spring onions, red pepper, feta chee...",mediterranean,"['couscous', 'spring', 'onions', 'pepper', 'fe..."
2,10-minute pad Thai,n,18572,n,3.9,5,5,31.0,Easy,Serves 2,A delicious one-pan dinner - who needs takeaways?,"200g ,raw prawns, small pack coriander,, stalk...","raw prawns, beansprouts, egg, juice 1 lime, fi...",thai,"['prawns', 'beansprouts', 'egg', 'lime', 'fish..."
3,10-minute steak & blue cheese wrap,n,15570,n,3.0,10,n,1.0,Easy,Serves 1,"A satisfying, meaty snack for a quick bite","1 tbsp ,olive oil, 1 x 140g/5oz ,sirloin steak...","olive oil, sirloin steak, small red onion, re...",american,"['olive', 'oil', 'steak', 'onion', 'pepper', '..."
4,10-minute steak & noodle supper,n,16166,n,3.0,10,n,3.0,Easy,Serves 2,A super-speedy beef and egg noodle dish – and ...,"200g ,rump steak, 200g ,pak choi, 1 ,red peppe...","pak choi, red pepper, sunflower oil,",chinese,"['pak', 'choi', 'pepper', 'sunflower', 'oil']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,Zingy salmon & brown rice salad,healthy,12730,healthy,4.0,15,25,43.0,Easy,Serves 3 - 4,This summery recipe has the ideal combination ...,"200g ,brown basmati rice, 200g ,frozen soya be...","brown basmati rice, frozen soya beans, salmon ...",british,"['basmati', 'rice', 'soya', 'beans', 'salmon',..."
10669,Zombie fingers,n,15405,n,4.0,30,n,4.0,Easy,Serves about 15,These freaky edible fingers are naturally swee...,"250g ,stoned dates, 100g ,dark chocolate,, cho...","stoned dates, dark chocolate, smooth peanut bu...",british,"['chocolate', 'peanut', 'butter', 'almonds']"
10670,Zoolbia (lacy saffron fritters),vegetarian,4140,vegetarian,n,5,25,n,Easy,Makes 14-16,Make these moreish Persian fritters for a cele...,"175g ,plain flour, 1 x 7g sachet ,fast-acting...","plain flour, icing sugar,",persian,"['flour', 'sugar']"
10671,‘Butter pie’ with apples & cheese,vegetarian,961,vegetarian,4.0,20,1 hr and 30,4.0,More effort,Cuts into 10,This hearty potato pie hails from Lancashire a...,"800g ,potato, (we used Desiree), sliced into 5...","unsalted butter, large (not mild) onions, smal...",english,"['butter', 'not', 'onions', 'thyme', 'coxs', '..."
